In [1]:
# This code calculates the vertical DEM shift using the Nuth and Kääb point cloud coregistration

import geoutils as gu
import numpy as np
import pandas as pd

import xdem

/Users/karinazikan/opt/anaconda3/envs/xdem-dev/lib/python3.11/site-packages/skgstat/models.py:22: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/Users/karinazikan/opt/anaconda3/envs/xdem-dev/lib/python3.11/site-packages/skgstat/models.py:86: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/Users/karinazikan/opt/anaconda3/envs/xdem-dev/lib/python3.11/site

In [6]:
# **files**
reference_dem = pd.read_csv("Sites/RCEW/IS2_Data/RCEW-ICESat2-ATL03-2.csv")
dem_to_be_aligned = xdem.DEM("Sites/RCEW/DEMs/RCEW_1m_WGS84UTM11_WGS84.tif")

/Users/karinazikan/opt/anaconda3/envs/xdem-dev/lib/python3.11/site-packages/geoutils/raster/raster.py:1407: UserWarning: Setting default nodata -99999 to mask non-finite values found in the array, as no nodata value was defined.
  warnings.warn(


In [11]:
reference_dem

,time,E,N,z
0,2019-05-27 12:28:42.743,523430.475310,4.772917e+06,1900.3315
1,2019-05-27 12:28:42.744,523429.877709,4.772922e+06,1898.8876
2,2019-05-27 12:28:42.745,523429.578021,4.772925e+06,1898.0692
3,2019-05-27 12:28:42.745,523429.435821,4.772927e+06,1898.5760
4,2019-05-27 12:28:42.745,523429.355678,4.772927e+06,1897.7479
...,...,...,...,...
1409899,2022-08-20 04:06:51.403,513872.015151,4.795333e+06,1795.9928
1409900,2022-08-20 04:06:51.403,513871.930533,4.795334e+06,1795.6282
1409901,2022-08-20 04:06:51.403,513871.860290,4.795335e+06,1795.5538
1409902,2022-08-20 04:06:51.403,513871.792962,4.795335e+06,1795.6029


In [8]:
# **rename columns to E, N, & z if necisary**  
reference_dem = reference_dem.rename(columns={'X': 'E','Y': 'N','Z':'z'})

In [10]:
# **Nuth and Kääb coregistration**

nuth_kaab = xdem.coreg.NuthKaab()

nuth_kaab.fit_pts(reference_dem, dem_to_be_aligned, mask_high_curv=True, verbose = True)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Running Nuth and Kääb (2011) coregistration. Shift pts instead of shifting dem
   Calculate slope and aspect


/Users/karinazikan/opt/anaconda3/envs/xdem-dev/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,


   Statistics on initial dh:
      Median = nan - NMAD = nan
   Iteratively estimating horizontal shit:


   Progress:   0%|                     | 0/10 [00:00<?, ?it/s]


AssertionError: 

In [ ]:
aligned_dem = nuth_kaab.apply(dem_to_be_aligned)